# Lesson 8 - Hierarchically Chaining the Agent Calls using a Router Agent

You will now create a hierarchical workflow using a router agent. Instead of having a fixed linear workflow like in the previous lesson, you will use a third agent (router) in the client-side that will decide when to call each ACP agent.

## 8.1. Start Up both ACP Servers

First make sure the Insurer server is still running:
- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, then you don't need to do anything else.
- If the agent has stopped running (the lab environment resets after 120 min), then you can run the server again by typing:
  - `uv run crew_agent_server.py`


**Note**: If you see this warning: 
`WARNING: Can not reach server, check if running on http://127.0.0.1:8333 : Request failed after 5 retries`
you can ignore it. You'll learn later in another lesson about the BeeAI platform, which a registry you can use to manage and discover agents. If the platform is installed, it runs by default on port 8333. The ACP servers are configured to automatically connect to the platform. Since the platform is not installed in this environment, the ACP server will generate a warning.

Also make sure the Hospital server is still running:
- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, then you don't need to do anything else.
- If the agent has stopped running (the lab environment resets after 120 min), then you can run the server again by typing:
  - `uv run smolagents_server.py`

## 8.2. Import ACPCallingAgent

The router agent is already implemented for you as the `ACPCallingAgent`. You are provided with a python file called `fastacp.py` where you can find the definition of the `ACPCallingAgent`. 

In [1]:
import asyncio

import nest_asyncio
from acp_sdk.client import Client
from colorama import Fore
from fastacp import ACPCallingAgent, AgentCollection
from smolagents import LiteLLMModel

In [2]:
print(ACPCallingAgent.__doc__)


    This agent uses JSON-like ACP agent calls, similarly to how ToolCallingAgent uses tool calls,
    but directed at remote ACP agents instead of local tools.

    Args:
        acp_agents (`dict[str, Agent]`): ACP agents that this agent can call.
        model (`Callable[[list[dict[str, str]]], ChatMessage]`): Model that will generate the agent's actions.
        prompt_templates ([`Dict[str, str]`], *optional*): Prompt templates.
        planning_interval (`int`, *optional*): Interval at which the agent will run a planning step.
        **kwargs: Additional keyword arguments.
    


## 8.3. Run the Hierarchical Workflow 

In [3]:
nest_asyncio.apply()

**Note**: The `fastacp.py` file does not only contain the definition for the ACPCallingAgent, but it also includes this method: `AgentCollection.from_acp` where the client objects (`insurer` and `hospital`) discover the agents hosted on their corresponding servers by calling the method `.agents()`.

In [5]:
model = LiteLLMModel(model_id="openrouter/google/gemma-3-12b-it:free")


async def run_hospital_workflow() -> None:
    async with (
        Client(base_url="http://localhost:8000") as hospital,
        Client(base_url="http://localhost:8001") as insurer,
    ):
        # agents discovery
        agent_collection = await AgentCollection.from_acp(insurer, hospital)
        acp_agents = {
            agent.name: {"agent": agent, "client": client}
            for client, agent in agent_collection.agents
        }
        print(acp_agents)
        # passing the agents as tools to ACPCallingAgent
        acpagent = ACPCallingAgent(acp_agents=acp_agents, model=model)
        # running the agent with a user query
        result = await acpagent.run(
            "do i need rehabilitation after a shoulder reconstruction"
            " and what is the waiting period from my insurance?"
        )
        print(Fore.YELLOW + f"Final result: {result}" + Fore.RESET)

In [ ]:
asyncio.run(run_hospital_workflow())

**Optional Reading:** Here's how the hierarchical flow works using the provided file `fastacp.py`:

1. The agents hosted on each server are first discovered by their corresponding client objects and then converted to tools for the router agent (ACPCallingAgent): 
  
  <img src="hr_1.png" width="650">

2. When the router agent receives a user query, it breaks downs the query into smaller steps where each step can be executed by the specialized agent. For a given step, the router agent uses the client of the specialized agent to send the request to it:
  
  <img src="hr_2.png" width="650">

## 8.4. Resources

- [Compose Agents](https://agentcommunicationprotocol.dev/how-to/compose-agents)
- [Chained Agents as a Python file](https://github.com/nicknochnack/ACPWalkthrough/blob/main/5.%20Chained%20Agents.py)